In [ ]:
import pandas as pd
import numpy as np
import itertools
from datetime import datetime

In [ ]:
pp_dep = pd.read_csv("pp_dep.csv", sep=";", dtype={"reg":str}, parse_dates=['period_date'])

In [ ]:
pp_dep.period_date.unique()

In [ ]:
pp_dep.indicateur.unique()

In [ ]:
corres = {
    "Nombre de dossiers MaPrimeRénov acceptés" : 
        ["Nombre de dossiers MaPrimeRénov' payés - MPR"],
    "Nombre de bâtiments de l’Etat retenus pour faire l’objet d’une rénovation énergétique" : 
        ['Nombre de bâtiments Etat dont le marché de rénovation est notifié - RBE'],
    "Nombre de bonus écologiques et de primes à la conversion versés " : 
        ['Nombre de primes à la conversion - DVP'],
    "Nombre de projets industriels soutenus" : ['Nombre d’entreprises - FAA', 
                                                'Nombre d’entreprises bénéficiares - PIT', 
                                                'Nombre d’entreprises bénéficiaires - SAC'],
    "Nombre de chèques export et chèques VIE (volontariat international en entreprise" : 
        ['Nombre d’entreprises bénéficiares - SBF'],
    "Nombre de cathédrales et de monuments nationaux retenus pour faire l’objet d’une rénovation" : 
        ['Nombre de projets de rénovation de cathédrales et de monuments nationaux initiés - PAE'],
    "Nombre de monuments appartenant à des collectivités territoriales ou à des acteurs privés retenus pour faire l’objet d’une rénov" : 
        ['Nombre de projets de rénovation de monuments historiques appartenant aux collectivités territoriales bénéficiaires initiés - PAT'],
    "Nombre de contrats d’apprentissage ayant bénéficié de l’aide exceptionnelle " : 
        ['Nombre de contrats d’apprentissage bénéficiaires de l’aide exceptionnelle - APP'],
    "Nombre d’aides versées pour l’embauche des jeunes" : 
        ["Nombre d'aides à l'embauche des jeunes - PEJ"],
    "Nombre de jeunes entrés en CIE (contrats initiative emploi) et PEC (parcours emploi compétences) " : 
        ['Entrées de jeunes en CIE - CIE', 'Entrées de jeunes en PEC - PEC']
}

In [ ]:
pd.DataFrame.from_dict(corres, orient='index').to_csv("correspondances.csv")

In [ ]:
list_indicateurs = list(itertools.chain.from_iterable(corres.values()))
len(list_indicateurs)

In [ ]:
pp_dep_indics = pp_dep[pp_dep['indicateur'].isin(list_indicateurs)].reset_index(drop=True)
pp_dep_indics.drop('Unnamed: 0', axis=1, inplace=True)
pp_dep_indics.shape

In [ ]:
pp_dep.loc[pp_dep.indicateur == "Nombre de PME - FAA"].valeur.unique()

In [ ]:
def get_latest_date_for_indic_and_dep(indicateur, dep):
    df = pp_dep_indics[(pp_dep_indics['indicateur'] == indicateur) & 
                          (pp_dep_indics['dep'] == dep)]
    return df['period_date'].max()


pp_dep_to_export = pp_dep_indics.loc[pp_dep_indics.apply(lambda x: get_latest_date_for_indic_and_dep(x['indicateur'], x['dep']) == x['period_date'], axis=1)]
pp_dep_to_export.reset_index(drop=True, inplace=True)
pp_dep_to_export.head()

In [ ]:
export = pd.pivot_table(pp_dep_to_export, columns=['dep'], index=['mesure', 'indicateur'])
export.columns = export.columns.get_level_values(1)
export['National'] = export.sum(axis=1)
export.reset_index(drop=False, inplace=True)
export.set_index('indicateur', inplace=True,)

In [ ]:
export

In [ ]:
# old
export

In [ ]:
# Ajout du nom des départements
deps = pp_dep_to_export.groupby(['dep', 'departement']).count().reset_index()[['dep', 'departement']]
dep2names = {dep: dep_name for dep, dep_name in zip(deps['dep'], deps['departement'])}
dep_names = pd.DataFrame(data=[[dep2names.get(dep, '-') for dep in export.columns]], columns=export.columns)
to_export = pd.concat([dep_names, export]).T.reset_index(drop=False)
to_export

In [ ]:
to_export[to_export['Nombre d’entreprises - FAA'] != 0]

In [ ]:
to_export.to_csv('export.csv', index=False, sep=";")

In [ ]:
l1 = list(to_export.columns)
[x for x in list_indicateurs if x not in l1]